In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('HouseData.csv')

In [4]:
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [5]:
df.shape

(13320, 9)

In [6]:
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [7]:
df1 = df.drop(['area_type','availability','society','balcony'], axis = 1)

In [8]:
df1.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [9]:
df1.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [10]:
df1.dropna(inplace = True)

In [11]:
df1['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [12]:
df1['BHK'] = df1['size'].apply(lambda x: int(x.split()[0]))

In [13]:
df1.drop(['size'], axis = 1, inplace = True)

In [14]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [15]:
def convert_to_float(x):
    if is_float(x):
        return float(x)
    else:
        try:
            return (float(x.split('-')[0]) + float(x.split('-')[1]))/2
        except:
            return None

In [16]:
df1['total_sqft'] = df1['total_sqft'].apply(convert_to_float)

In [17]:
location_stats = df1.location.apply(lambda x: x.strip()).value_counts()

In [18]:
location_stats_less_than_10 = location_stats[location_stats < 10]

In [19]:
df1.location = df1.location.apply(lambda x: 'Other' if x in location_stats_less_than_10 else x)

In [20]:
df1['price_per_sqft'] = round(df1['price']*100000/df1['total_sqft'],2)

In [21]:
df1.head()

,location,total_sqft,bath,price,BHK,price_per_sqft
0,Electronic City Phase II,1056.0,2.0,39.07,2,3699.81
1,Chikka Tirupathi,2600.0,5.0,120.00,4,4615.38
2,Uttarahalli,1440.0,2.0,62.00,3,4305.56
3,Lingadheeranahalli,1521.0,3.0,95.00,3,6245.89
4,Kothanur,1200.0,2.0,51.00,2,4250.00


In [23]:
df1[df1['total_sqft']/df1['BHK'] < 300].head()

,location,total_sqft,bath,price,BHK,price_per_sqft
9,Other,1020.0,6.0,370.0,6,36274.51
45,HSR Layout,600.0,9.0,200.0,8,33333.33
58,Murugeshpalya,1407.0,4.0,150.0,6,10660.98
68,Devarachikkanahalli,1350.0,7.0,85.0,8,6296.30
70,Other,500.0,3.0,100.0,3,20000.00


In [24]:
df2 = df1[~(df1['total_sqft']/df1['BHK'] < 300)]

,location,total_sqft,bath,price,BHK,price_per_sqft
